# [Workshop 5. Assessment of Survey Data Quality](https://wapor.org/events/annual-conference/current-conference/training-workshops/)

# Part II: Design effect and weighting

Author: <a href="mailto:alexander.seymer@plus.ac.at?subject=Regarding the WAPOR 2023 workshop">Alexander Seymer @ PLUS</a>

Date: September 19, 2023

## Abstract

In this session of the workshop, the design effect, as measure for the impact of the sampling design on the variance of an estimator, and weighting as tool to account for complex sampling designs will be discussed.

## Preparing the R session


In [11]:
source("install.R")

## Weighting

Although we will use R, alternative software packages provide means to employ weights:

| Software | Reference |
| -------- | --------- |
| Python | Paczkowski, W. R. (2022). Modern survey analysis: Using Python for deeper insights. Springer Nature. [https://link.springer.com/book/10.1007/978-3-030-76267-4](https://link.springer.com/book/10.1007/978-3-030-76267-4) | 
| SAS | Lewis, T. H. (2016). [Complex survey data analysis with SAS. CRC Press.](https://www.routledge.com/Complex-Survey-Data-Analysis-with-SAS/Lewis/p/book/9781498776776) |
| SPSS | Zou, D., Lloyd, J. E. V., & Baumbusch, J. L. (2019). Using SPSS to Analyze Complex Survey Data: A Primer. Journal of Modern Applied Statistical Methods, 18. [http://jmasm.com/index.php/jmasm/article/view/1026](http://jmasm.com/index.php/jmasm/article/view/1026) | 
| STATA | Weighing Data in Stata—Stata Help—Reed College. (2014). Retrieved September 7th, 2023, from [https://www.reed.edu/psychology/stata/gs/tutorials/weights.html](https://www.reed.edu/psychology/stata/gs/tutorials/weights.html) |


Complex sampling designs for surveys are commonly applied by employing:

- stratification (divide the population in homogenous groups and sample from each group a specific number);
- clustering (divide the population in groups, e.g. regions, and sample from a random subset of this groups);
- unequal sampling (oversampling of subgroups of interest).

Considering these samples as random samples will result in biased standard errors. 

A commmon approach to account for the bias is providing design weights. Most surveys using complex sampling strategies are providing weights to adjust for the deviation from the random sampling.


### Example 1: Weighting Values in Crisis data for Austrian Sample

Weighting requires information about the target population or the inferential population. In this example, official statistics from [EUROSTAT](https://ec.europa.eu/eurostat/web/main/home) will be used as reference for the inferential population including sex, age, education and region. Hence, we will:

1. Import data from EUROSTAT
2. Prepare reference points for weighting
3. Get Values in Crisis Data
4. Apply Weighting
5. Assess results


### Example 1.1 Import data from EUROSTAT

We will start by searching the EUROSTAT database for the relevant data. Let's check all database with containing the phrase `Population` in the title

In [12]:
search_eurostat("Population") %>%
  datatable(filter = "top")

HTML widgets cannot be represented in plain text (need html)

Let's restrict the results to also containing `Region`.

In [13]:
search_eurostat("Population") %>%
    filter(grepl("Region",title, ignore.case=TRUE)) %>%
      datatable(filter = "top")

HTML widgets cannot be represented in plain text (need html)

Ok, now add `Education`.

In [14]:
search_eurostat("Population") %>%
    filter(grepl("Region",title, ignore.case=TRUE)) %>%
    filter(grepl("Education",title, ignore.case=TRUE)) %>%
      datatable(filter = "top")

HTML widgets cannot be represented in plain text (need html)

In [15]:
EuroDataAT <- get_eurostat_json(
  "lfst_r_lfsd2pop",
  filters = list(
    geo = c("AT", paste0("AT",c(11:13,21,22,31:34))),
    time = 2020
  ))

datatable(EuroDataAT,
         filter = "top")

HTML widgets cannot be represented in plain text (need html)

### Example 1.2 Prepare reference points for weighting

#### Sex

To subset the sex distribution, we will use the `TOTAL` column from education (`isced11`), the older than or equal to 15 years (`Y_GE15`) from age, remove totals (`T`) from sex and use only `AT` for regioin (`geo`).

In [16]:
# Subset sex distribution
DatGndr <- EuroDataAT %>%
  filter(isced11 == "TOTAL") %>%
  filter(age == "Y_GE15") %>%
  filter(sex != "T") %>%
  filter(geo == "AT") %>%
  dplyr::select(sex, values) %>%
  setNames(c("Category", "Count"))

# Calculate share
DatGndr$Share <- DatGndr$Count/sum(DatGndr$Count)

datatable(DatGndr) %>%
    formatPercentage(3) %>% 
    formatStyle(3, background = styleColorBar(c(0,1), 'lightblue'),
                         backgroundSize = '98% 88%',
                         backgroundRepeat = 'no-repeat',
                         backgroundPosition = 'center')

HTML widgets cannot be represented in plain text (need html)

#### Age

A limitation for the application of age is that we need to use the predefined groups by official statistics. Otherwise, we use the total count (`T`) of sex, the specific age groups and apply the same filter for education and region as for sex.

In [17]:
# Subset age group distribution
DatAge <- EuroDataAT %>%
  filter(isced11 == "TOTAL") %>%
  filter(age %in% c("Y15-24","Y25-34","Y35-44","Y45-54","Y55-64","Y_GE65")) %>%
  filter(sex == "T") %>%
  filter(geo == "AT") %>%
  dplyr::select(age, values) %>%
  setNames(c("Category", "Count"))

# Calculate share
DatAge$Share <- DatAge$Count/sum(DatAge$Count)

datatable(DatAge) %>%
    formatPercentage(3) %>% 
    formatStyle(3, background = styleColorBar(c(0,1), 'lightblue'),
                         backgroundSize = '98% 88%',
                         backgroundRepeat = 'no-repeat',
                         backgroundPosition = 'center')

HTML widgets cannot be represented in plain text (need html)

#### Education

The limitation of existing categories becomes even bigger with education as the eurostat database provides only three categories with ISCED 0-2, ISCED 3-4 and ISCED 5-8. 

In [18]:
# Subset education distribution
DatEdu <- EuroDataAT %>%
  filter(isced11 %in% c("ED0-2", "ED3_4", "ED5-8")) %>%
  filter(age == "Y_GE15") %>%
  filter(sex == "T") %>%
  filter(geo == "AT") %>%
  dplyr::select(isced11, values) %>%
  setNames(c("Category", "Count"))

# Calculate share
DatEdu$Share <- DatEdu$Count/sum(DatEdu$Count)

datatable(DatEdu) %>%
    formatPercentage(3) %>% 
    formatStyle(3, background = styleColorBar(c(0,1), 'lightblue'),
                         backgroundSize = '98% 88%',
                         backgroundRepeat = 'no-repeat',
                         backgroundPosition = 'center')

HTML widgets cannot be represented in plain text (need html)

#### Region

In [19]:
# Subset NUTS2 distribution
DatReg <- EuroDataAT %>%
  filter(isced11 == "TOTAL") %>%
  filter(age == "Y_GE15") %>%
  filter(sex == "T") %>%
  filter(geo != "AT") %>%
  dplyr::select(geo, values) %>%
  setNames(c("Category", "Count"))

# Relabel
DatReg$Category <- DatReg$Category %>%
  str_replace("AT11","Burgenland") %>%
  str_replace("AT12","Lower Austria") %>%
  str_replace("AT13","Vienna") %>%
  str_replace("AT21","Carinthia") %>%
  str_replace("AT22","Styria") %>%
  str_replace("AT31","Upper Austria") %>%
  str_replace("AT32","Salzburg") %>%
  str_replace("AT33","Tyrol") %>%
  str_replace("AT34","Vorarlberg")

# Calculate share
DatReg$Share <- DatReg$Count/sum(DatReg$Count)

datatable(DatReg) %>%
    formatPercentage(3) %>% 
    formatStyle(3, background = styleColorBar(c(0,1), 'lightblue'),
                         backgroundSize = '98% 88%',
                         backgroundRepeat = 'no-repeat',
                         backgroundPosition = 'center')

HTML widgets cannot be represented in plain text (need html)

#### Import Data from Survey

The current data is from October 14, 2021 available at AUSSDA: [doi:10.11587/LIHK1L](https://data.aussda.at/dataset.xhtml?persistentId=doi:10.11587/LIHK1L)

We will use the following variables from the VIC data:

| Construct | Variablename | 
|:----- |:------ |
| Sex | Q01 | 
| Age | Q02_age_grouped |
| Education | Q05_ISCED_3_Levels |
| Region | Q09_AUT |


In [20]:
VIC_data <- readRDS("data/10742_da01_en_v2_0.Rdata") %>%
    filter(country == 1) %>% # 1 for Austria
    # Subset VIC data
    dplyr::select(ID_merge,Q01, Q02_age_grouped, Q05_ISCED_3_Levels, Q09_AUT)     

Let's investigate the distribution and attributes of these variables in the VIC data.

In [21]:
table1(~ as_factor(Q01) + 
         as_factor(Q02_age_grouped) + 
         as_factor(Q05_ISCED_3_Levels) + 
         as_factor(Q09_AUT),
       data = VIC_data) %>%
  display_html()

,Overall(N=2018)
Gender,
Male,992 (49.2%)
Female,1023 (50.7%)
Diverse,0 (0%)
Missing,3 (0.1%)
Age - grouped,
14-19 years,121 (6.0%)
20-24 years,169 (8.4%)
25-29 years,166 (8.2%)
30-34 years,165 (8.2%)


#### Align both data sources

We will need align the labels and coding from both sources. We will use the EUROSTAT data as point of reference here and subset the VIC data to manipulate the labels and coding.

##### Sex

For the sex variable, we simply overwrite the labels.

In [22]:
# Make it a factor and drop empty levels
VIC_data$Q01 <- VIC_data$Q01 %>%
  as_factor() %>%
  factor()

# Set factor labels of VIC data to EUROSTAT labels.
levels(VIC_data$Q01) <- DatGndr$Category

###### Age

For age, we need to regroup the variable slighly.

In [23]:
# Make it a factor and drop empty levels
VIC_data$Q02_age_grouped <- VIC_data$Q02_age_grouped %>%
  as_factor() %>%
  factor()

# Get original categories from VIC data
old_labs <- levels(VIC_data$Q02_age_grouped)

# Create new variable 
VIC_data$age_grouped <- NA
VIC_data$age_grouped[VIC_data$Q02_age_grouped %in% old_labs[1:2]] <- DatAge$Category[1]
VIC_data$age_grouped[VIC_data$Q02_age_grouped %in% old_labs[3:4]] <- DatAge$Category[2]
VIC_data$age_grouped[VIC_data$Q02_age_grouped %in% old_labs[5:6]] <- DatAge$Category[3]
VIC_data$age_grouped[VIC_data$Q02_age_grouped %in% old_labs[7:8]] <- DatAge$Category[4]
VIC_data$age_grouped[VIC_data$Q02_age_grouped %in% old_labs[9:10]] <- DatAge$Category[5]
VIC_data$age_grouped[VIC_data$Q02_age_grouped %in% old_labs[11]] <- DatAge$Category[6]

# Make the new variable a ordered factor
VIC_data$age_grouped <- factor(VIC_data$age_grouped,
                              levels = DatAge$Category,
                              labels = DatAge$Category,
                              ordered = TRUE)

# A quick check if the recoding worked
table(VIC_data$Q02_age_grouped,
     VIC_data$age_grouped)

# Drop old variable
VIC_data <- VIC_data %>%
    select(!Q02_age_grouped)

                    
                     Y15-24 Y25-34 Y35-44 Y45-54 Y55-64 Y_GE65
  14-19 years           121      0      0      0      0      0
  20-24 years           169      0      0      0      0      0
  25-29 years             0    166      0      0      0      0
  30-34 years             0    165      0      0      0      0
  35-39 years             0      0    153      0      0      0
  40-44 years             0      0    174      0      0      0
  45-49 years             0      0      0    171      0      0
  50-54 years             0      0      0    191      0      0
  55-59 years             0      0      0      0    199      0
  60-64 years             0      0      0      0    148      0
  65 years and older      0      0      0      0      0    361

##### Education

Education is already coded as need, hence we just need to relabel the variable.

In [24]:
# Make it a factor and drop empty levels
VIC_data$Q05_ISCED_3_Levels <- VIC_data$Q05_ISCED_3_Levels %>%
  as_factor() %>%
  factor()

# Set factor labels of VIC data to EUROSTAT labels.
levels(VIC_data$Q05_ISCED_3_Levels) <- DatEdu$Category

##### Region

In [25]:
# Make it a factor and drop empty levels
VIC_data$Q09_AUT <- VIC_data$Q09_AUT %>%
  as_factor() %>%
  factor()

##### Rename variables

The variable names need to be identical to the later target definition.

In [26]:
colnames(VIC_data) <- colnames(VIC_data) %>%
    str_replace("Q01","Sex") %>%
    str_replace("age_grouped", "Age") %>%
    str_replace("Q05_ISCED_3_Levels", "Education") %>%
    str_replace("Q09_AUT","Region")

### Weighting

The procedure uses the [anesrake](http://cran.r-project.org/web/packages/anesrake/index.html) package by Pasek (2018) and follows the blog entry on raking weights by Daze (2012).

- Pasek, Josh. 2018. Anesrake: ANES Raking Implementation. https://CRAN.R-project.org/package=anesrake.- Daza, Sebastian. 2012. “https://sdaza.com/blog/2012/raking/.” Raking Weights with R. https://sdaza.com/blog/2012/raking/


We will perform three steps:

1. Define a target list
2. Run the raking itself
3. Assess the output


##### Define a target list

The anesrake package requires a target list consisted of named vectors with the marginal proportions. Hence, we will create this list first:

In [27]:
# Prepare list of population marginal proportions
target <- list(
  "Sex" = DatGndr$Share %>%
    setNames(DatGndr$Category),
  "Age" = DatAge$Share %>%
    setNames(DatAge$Category),
  "Education" = DatEdu$Share %>%
    setNames(DatEdu$Category),
  "Region" = DatReg$Share %>%
    setNames(DatReg$Category)
)

In [28]:
target$Sex

M         F 
0.4873916 0.5126084

#### Run the raking function

As raking is an iterative procedure, we need to set the seed of the CPU to receive reproducible results.

In [31]:
set.seed(19092023)


RakingResult <- anesrake(inputter = target,
                         dataframe = VIC_data,
                         caseid = VIC_data$ID_merge,
                         cap = 10,
                         type = "nolim",
                         force1 = FALSE)
    

Warning message in anesrake(inputter = target, dataframe = VIC_data, caseid = VIC_data$ID_merge, :
"Targets for Sex do not sum to 100%. Did you make a typo entering the targets?"
Warning message in anesrake(inputter = target, dataframe = VIC_data, caseid = VIC_data$ID_merge, :
"You can force variables to sum to 1 by setting force1 to 'TRUE'"


ERROR: Error in x + weights: nicht-numerisches Argument für binären Operator


In [ ]:
# Weighting

The procedure uses the [anesrake](http://cran.r-project.org/web/packages/anesrake/index.html) package by @anesrake for R and follows the blog entry on raking weights by @daza_2012.



## Subset the data by wave and run raking

```{r}
# Subset data relevant for weighting by wave
subsamples <- c("Teilnahme_W1",
                "Teilnahme_W2",
                "Teilnahme_W3",
                "Teilnahme_W12",
                "Teilnahme_W13",
                "Teilnahme_W23",
                "Teilnahme_W123")

Raking <- lapply(subsamples, 
  function(x){
    if(grepl("W3",x)){ # If Wave 3 use Wave 3 info on person
      subset_vars <- c("ID_U0", # ID
                       "VIC3_Q1", # Sex
                       "Age_W3", # Age groups
                       "Edu_W3", # Education
                       "VIC3_Q8") # Region
      subset_target <- target22
    } else {
      if(grepl("W2",x)){ # If Wave 2 use Wave 2 info on person
        subset_vars <- c("ID_U0", # ID
                         "VIC2_Q1", # Sex
                         "Age_W2", # Age groups
                         "Edu_W2", # Education
                         "VIC2_Q8") # Region
        subset_target <- target21
      } else { # Only for Wave 1 use Wave 1 info on person
        subset_vars <- c("ID_U0", # ID
                         "VIC1_Q1", # Sex
                         "Age_W1", # Age groups
                         "Edu_W1",# Education
                         "VIC1_Q4") # Region
        subset_target <- target20
      }
    }
    
    # Subset data
    Weight_Data <- VIC_data %>%
      .[.[,x] == 1,] %>%
      filter(is.na(ID_U0) == FALSE) %>%
      select(all_of(subset_vars)) %>%
      set_names(c("ID_U0",
                  "Geschlecht", 
                  "Alter",
                  "Bildung",
                  "County"))
    
    # Align variable labels of both data for
    # County:
    Weight_Data$County <- as_factor(Weight_Data$County) %>%
      factor(levels = DatCnty21$Category)

    DatWave <- Weight_Data %>% as.data.frame()
    
    set.seed(21072021)
    
    RakingResult <- anesrake(subset_target,
                             DatWave,
                             DatWave$ID_U0,
                             cap = 10,
                             type = "nolim",
                             force1 = FALSE)
    
    return(list(
      DatWave = DatWave,
      RakingResult = RakingResult
    ))
  }) %>%
  setNames(subsamples)
```

## Print some summary statistics:

### Wave 1 weights

```{r}
summary(Raking$Teilnahme_W1$RakingResult)
```

### Wave 2 weights

```{r}
summary(Raking$Teilnahme_W2$RakingResult)
```

### Wave 3 weights

```{r}
summary(Raking$Teilnahme_W3$RakingResult)
```

### Wave 1 + 2 (panel data) weights

```{r}
summary(Raking$Teilnahme_W12$RakingResult)
```

### Wave 1 + 3 (panel data) weights

```{r}
summary(Raking$Teilnahme_W13$RakingResult)
```

### Wave 2 + 3 (panel data) weights

```{r}
summary(Raking$Teilnahme_W23$RakingResult)
```

### Wave 1 + 2 + 3 (panel data) weights

```{r}
summary(Raking$Teilnahme_W123$RakingResult)
```

# Export results to SPSS

```{r, message=FALSE, warning=FALSE}
names(Raking) <- names(Raking) %>%
  str_remove_all("_") %>%
  str_remove_all("Teilnahme")

lapply(names(Raking), function(subname){
  data.frame(
    Raking[[subname]]$RakingResult$caseid,
    Raking[[subname]]$RakingResult$weightvec) %>%
    setNames(c("U0", paste0("Weight_",subname))) %>%
    write_sav(here(paste0("export/",subname,"_weights.sav"))
  )
}) %>%
  invisible()
```

```{r, include=FALSE, echo=FALSE}
lapply(names(Raking), function(subname){
  file.copy(
    from = here(paste0("export/",subname,"_weights.sav")),
    to = here(paste0("../",subname,"_weights.sav")),
          overwrite = TRUE)
})
```

# Storage key data

```{r}
list(DatAge20,  DatAge21,  DatAge22,
     DatCnty20, DatCnty21, DatCnty22,
     DatEdu,
     DatGndr20, DatGndr21, DatGndr22,
     target20,  target21,  target22) %>%
saveRDS(here("data/targets_w123.rda"))

saveRDS(Raking, here("data/Raking_w123.rda"))
Raking <- readRDS(here("data/Raking_w123.rda"))
```

# Descriptives of weighting variables

```{r}
lapply(Raking, function(x){
  desc_data <- psych::describe(x$RakingResult$weightvec) %>%
    as.data.frame() %>%
    dplyr::select(n, sd) %>%
    c(summary(x$RakingResult)$weight.summary[c(2,3,5,1,6)],
      summary(x$RakingResult)$general.design.effect,.) %>%
    setNames(c("1st Quartil",
               "Median",
               "3rd Quartil",
               "Minimum",
               "Maximum",
               "Design effect",
               "n",
               "Standard deviation"
    )) 
  desc_data <- desc_data[c("n",
                           "Standard deviation",
                           "1st Quartil",
                           "Median",
                           "3rd Quartil",
                           "Minimum",
                           "Maximum",
                           "Design effect"
  )] %>%
    as.data.frame()
  # selection of weights bigger than 3
  big_sel <- grep(TRUE,(x$RakingResult$weightvec>3))
  desc_data$'No. of weights > 3 (W>3)' <- length(x$RakingResult$weightvec[big_sel]) 
  desc_data$'Sum of W>3' <- sum(x$RakingResult$weightvec[big_sel])
  return(desc_data)
}) %>% do.call("rbind",.) %>%
  setNames(c("n",
             "Standard deviation",
             "1st Quartil",
             "Median",
             "3rd Quartil",
             "Minimum",
             "Maximum",
             "Design effect",
             'No. of weights > 3 (W>3)',
             'Sum of W>3'
  )) %>%
  t() %>%
  as.data.frame() %>%
  cbind(rownames(.),.) %>%
  setNames(c("Parameter",colnames(.)[-1])) %>%
  flextable() %>%
  autofit() %>%
  colformat_double(digits = 3) %>%
  colformat_double(i = c(1,8), digits = 0)

```



# Plot weight distributions

```{r}
plotdata <- lapply(names(Raking), function(x){
  cbind(x,
        Raking[[x]]$RakingResult$weightvec) %>%
    as.data.frame(row.names = FALSE) %>%
    setNames(c("Sample","Weight"))
}) %>%
  do.call("rbind",.)

plotdata$Sample <- factor(plotdata$Sample,
                          levels = c("W1","W2","W3","W12",
                                     "W13","W23","W123"),
                          ordered = TRUE)

p1 <- plotdata %>%   
  ggplot(aes(as.numeric(Weight))) +
  geom_histogram(color="black", fill="white") +
  facet_grid(rows = vars(Sample)) +
  theme_minimal() +
  labs(x = "",
       y = " ") +
  scale_x_continuous(breaks = 1:10) +
  theme(legend.position="none",
        plot.margin=unit(c(0,0,-0.5,0), "cm"))

p2 <- plotdata %>%
  ggplot(aes(as.numeric(Weight))) +
  geom_boxplot(width=0.6) +
  facet_grid(rows = vars(Sample)) +
  theme_minimal() +
  labs(x = "Weight",
       y = "") +
  #coord_cartesian(ylim = c(-0.4, 0.4)) +
  scale_y_continuous(limits = c(-0.5,0.5)) +
  scale_x_continuous(breaks = 1:10) +
  theme(legend.position="none",
        #axis.text.x=element_blank(),
        #axis.ticks.x=element_blank(),
        plot.margin=unit(c(0.25,0,0,0), "cm"),
        strip.text.x = element_blank())

p3 <- egg::ggarrange(p2, p1, widths  = 2:1)

ggsave(here::here("export/weights_plot_2022.png"),
       p3,
       dpi = 300)
```


### Design effect

And as with all biases, the first and foremost question is how serious is the issue? And of course, the answer is complicated and depends on different factors. Calculating the design effect helps to get an understanding of the gravity of the issue. The 
